# 💧 Ecuación de Bernoulli: Sistema de Bombeo de Agua
## Análisis gráfico e interactivo para ingeniería

---

### 📚 Introducción

La **Ecuación de Bernoulli generalizada** describe la conservación de energía en un fluido en movimiento. Para un sistema de bombeo que lleva agua desde un estanque inferior hasta un punto elevado, la ecuación se expresa como:

$$\frac{P_1}{\rho g} + \frac{v_1^2}{2g} + z_1 + H_b = \frac{P_2}{\rho g} + \frac{v_2^2}{2g} + z_2 + h_f$$

Donde:
- $P / \rho g$ → **Altura de presión** [m]
- $v^2 / 2g$ → **Altura cinética** [m]
- $z$ → **Altura potencial** o cota [m]
- $H_b$ → **Altura de la bomba** (energía aportada) [m]
- $h_f$ → **Pérdidas de carga** (fricción y menores) [m]

---

### 🔧 Condiciones del problema

| Punto | Descripción | Presión | Velocidad |
|-------|-------------|---------|----------|
| **1** | Superficie del estanque (aguas abajo) | $P_1 = P_{atm}$ | $v_1 \approx 0$ |
| **2** | Salida en el cerro (descarga libre) | $P_2 = P_{atm}$ | $v_2 = v$ |

Simplificando (presiones manométricas = 0 en ambos extremos, $z_1 = 0$):

$$\boxed{H_b = z_2 + \frac{v_2^2}{2g} + h_f}$$

> **Interpretación:** La bomba debe suministrar energía suficiente para vencer la diferencia de altura, la energía cinética del flujo y las pérdidas de carga en la tubería.

In [ ]:
# ============================================================
# 📦 IMPORTACIÓN DE LIBRERÍAS
# ============================================================
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.patches as patches
from matplotlib.patches import FancyArrowPatch, FancyBboxPatch
from ipywidgets import interact, FloatSlider, Layout
import warnings
warnings.filterwarnings('ignore')

# Constantes físicas
g = 9.81        # aceleración de gravedad [m/s²]
rho = 1000.0    # densidad del agua [kg/m³]

print("✅ Librerías cargadas correctamente")
print(f"   g = {g} m/s²  |  ρ_agua = {rho} kg/m³")

---
## 🎛️ Simulación Interactiva

Ajusta los parámetros con los **sliders** y observa cómo cambian:
1. El **diagrama físico** del sistema (estanque → bomba → cerro)
2. La **Línea de Energía Total** (EGL: *Energy Grade Line*)
3. La **Línea Piezométrica** (HGL: *Hydraulic Grade Line*)
4. La **distribución de energías** en cada punto del sistema

> 💡 **Tip:** Observa cómo la bomba eleva la línea de energía y cómo las pérdidas de carga la reducen gradualmente.

In [ ]:
# ============================================================
# 🖼️ FUNCIÓN PRINCIPAL DE VISUALIZACIÓN
# ============================================================

def graficar_bernoulli(z2=30.0, v2=2.0, D_mm=100.0, hf=5.0):
    """
    Visualiza el sistema de bombeo con la ecuación de Bernoulli.
    
    Parámetros:
    -----------
    z2   : Altura del cerro (cota de llegada) [m]
    v2   : Velocidad del flujo en la tubería [m/s]
    D_mm : Diámetro de la tubería [mm]
    hf   : Pérdidas de carga totales [m]
    """
    
    # ----------------------------------------------------------
    # 🔢 CÁLCULOS HIDRÁULICOS
    # ----------------------------------------------------------
    D = D_mm / 1000.0                        # diámetro en metros
    A = np.pi * D**2 / 4                     # área transversal [m²]
    Q = A * v2                               # caudal [m³/s]
    Q_lps = Q * 1000                         # caudal [L/s]
    
    # Alturas de presión (manométricas)
    P1_man = 0.0   # superficie estanque: presión manométrica = 0
    P2_man = 0.0   # descarga libre: presión manométrica = 0
    
    # Cotas
    z1 = 0.0       # referencia: estanque en z=0
    
    # Velocidades
    v1 = 0.0       # superficie del estanque: v ≈ 0
    
    # Alturas cinéticas
    hv1 = v1**2 / (2 * g)    # [m]
    hv2 = v2**2 / (2 * g)    # [m]
    
    # Altura de la bomba (Ecuación de Bernoulli generalizada)
    Hb = z2 + P2_man + hv2 + hf - (z1 + P1_man + hv1)
    
    # Potencia hidráulica de la bomba
    Pb = rho * g * Q * Hb    # [W]
    Pb_kW = Pb / 1000        # [kW]
    
    # Líneas de energía (EGL = z + P/ρg + v²/2g)
    # Puntos: 0=estanque, 1=entrada bomba, 2=salida bomba, 3=llegada cerro
    # Posición horizontal aproximada de la bomba: 15% del recorrido
    x_total = 10.0  # unidades horizontales del diagrama
    x_bomba = 1.5
    
    # EGL en cada punto clave
    EGL_0 = z1 + P1_man + hv1                    # estanque
    EGL_1 = EGL_0 - hf * 0.15                    # antes de bomba (pérdidas succión ~15%)
    EGL_2 = EGL_1 + Hb                            # después de bomba (ganancia)
    EGL_3 = z2 + P2_man + hv2                    # llegada al cerro
    
    # HGL = EGL - v²/2g
    HGL_0 = EGL_0 - hv1
    HGL_1 = EGL_1 - hv2   # en tubería ya tiene velocidad
    HGL_2 = EGL_2 - hv2
    HGL_3 = EGL_3 - hv2
    
    # ----------------------------------------------------------
    # 🎨 CONFIGURACIÓN DEL GRÁFICO
    # ----------------------------------------------------------
    fig = plt.figure(figsize=(16, 12))
    fig.patch.set_facecolor('#f8f9fa')
    
    # Layout: diagrama físico (izquierda) + gráfico energético (derecha)
    ax1 = fig.add_axes([0.05, 0.38, 0.42, 0.55])   # diagrama físico
    ax2 = fig.add_axes([0.55, 0.38, 0.42, 0.55])   # gráfico energético
    ax3 = fig.add_axes([0.05, 0.04, 0.90, 0.28])   # tabla/barras de energía
    
    # ══════════════════════════════════════════════════════════
    # DIAGRAMA 1: ESQUEMA FÍSICO DEL SISTEMA
    # ══════════════════════════════════════════════════════════
    ax1.set_facecolor('#e8f4f8')
    ax1.set_xlim(-1, 11)
    ax1.set_ylim(-5, z2 + 8)
    ax1.set_title('📐 Esquema del Sistema de Bombeo', fontsize=13, fontweight='bold', pad=10)
    ax1.set_xlabel('Distancia horizontal (esquemática)', fontsize=10)
    ax1.set_ylabel('Cota z [m]', fontsize=10)
    ax1.grid(True, alpha=0.3, linestyle='--')
    
    # Terreno / perfil del cerro
    x_terreno = np.array([0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 0])
    z_terreno_raw = np.array([0, 0, 0.05, 0.15, 0.35, 0.6, 0.78, 0.9, 0.97, 1.0, -0.5, -0.5])
    z_terreno = z_terreno_raw * z2
    ax1.fill(x_terreno, z_terreno, color='#8B7355', alpha=0.6, label='Terreno')
    ax1.fill([0, 0, 2, 2], [-5, 0, 0, -5], color='#8B7355', alpha=0.6)
    
    # Pasto en la superficie
    ax1.fill_between([0, 2], [0, 0], [0.3, 0.3], color='#4CAF50', alpha=0.7)
    ax1.fill_between(x_terreno[:10], z_terreno[:10], z_terreno[:10] + 0.3, color='#4CAF50', alpha=0.7)
    
    # Estanque (punto 1)
    estanque_h = min(3.0, z2 * 0.1 + 1.5)
    ax1.add_patch(FancyBboxPatch((-0.8, -estanque_h), 1.8, estanque_h,
                                  boxstyle="square", linewidth=2,
                                  edgecolor='#1565C0', facecolor='#90CAF9', alpha=0.8))
    # Agua en estanque
    ax1.fill([-0.75, -0.75, 0.95, 0.95],
             [-estanque_h, -0.1, -0.1, -estanque_h],
             color='#2196F3', alpha=0.6)
    ax1.axhline(y=0, xmin=0.0, xmax=0.18, color='#2196F3', linewidth=3, alpha=0.8)
    ax1.annotate('Nivel\nlibre', xy=(-0.2, 0.3), fontsize=8, color='#1565C0', ha='center')
    ax1.annotate('Estanque\n(Punto 1)', xy=(-0.2, -estanque_h - 0.8), fontsize=8,
                 ha='center', color='#1565C0', fontweight='bold')
    ax1.annotate('$z_1 = 0$ m', xy=(-0.2, -estanque_h - 1.5), fontsize=8,
                 ha='center', color='#1565C0')
    
    # Tubería
    x_pipe = np.array([0.5, 1.5, 1.5, 2.0, 10.0])
    z_pipe = np.array([0, 0, -2, -2, z2])
    # Interpolación suave para el perfil del terreno
    x_pipe_full = np.linspace(2.0, 10.0, 100)
    z_pipe_full = np.interp(x_pipe_full, [2.0, 10.0], [-2, z2])
    
    pipe_width = max(0.15, D * 8)
    ax1.plot([0.5, 1.5], [0, 0], color='#424242', linewidth=5, zorder=5)
    ax1.plot([1.5, 1.5], [0, -2], color='#424242', linewidth=5, zorder=5)
    ax1.plot(np.concatenate([[1.5], x_pipe_full]),
             np.concatenate([[-2], z_pipe_full]),
             color='#424242', linewidth=5, zorder=5)
    
    # Flujo en tubería (flechas)
    for xi, zi in zip([4, 6, 8], [z2*0.2, z2*0.5, z2*0.8]):
        dx = 0.5; dz = (z2 - (-2)) / 8 * 0.5
        ax1.annotate('', xy=(xi + dx, zi + dz), xytext=(xi, zi),
                     arrowprops=dict(arrowstyle='->', color='#2196F3', lw=2))
    
    # BOMBA
    bx, bz = 1.5, -2.5
    bomba = plt.Circle((bx, bz), 0.5, color='#FF5722', zorder=10)
    ax1.add_patch(bomba)
    ax1.text(bx, bz, '⚙', fontsize=16, ha='center', va='center', zorder=11)
    ax1.annotate(f'BOMBA\n$H_b = {Hb:.1f}$ m\n$P = {Pb_kW:.2f}$ kW',
                 xy=(bx, bz - 0.7), fontsize=8, ha='center', fontweight='bold',
                 color='#BF360C',
                 bbox=dict(boxstyle='round,pad=0.3', facecolor='#FFCCBC', alpha=0.8))
    
    # Punto de llegada (cerro)
    ax1.plot(10, z2, 'o', color='#E91E63', markersize=10, zorder=10)
    ax1.annotate(f'Punto 2\n$z_2 = {z2:.0f}$ m',
                 xy=(10, z2), xytext=(8.5, z2 + 1.5),
                 fontsize=8, fontweight='bold', color='#880E4F',
                 arrowprops=dict(arrowstyle='->', color='#880E4F'),
                 bbox=dict(boxstyle='round,pad=0.3', facecolor='#FCE4EC', alpha=0.8))
    
    # Cota z1 y z2
    ax1.axhline(y=0, color='#FF9800', linestyle=':', linewidth=1.5, alpha=0.8)
    ax1.axhline(y=z2, color='#9C27B0', linestyle=':', linewidth=1.5, alpha=0.8)
    ax1.annotate('', xy=(10.5, z2), xytext=(10.5, 0),
                 arrowprops=dict(arrowstyle='<->', color='#9C27B0', lw=2))
    ax1.text(10.7, z2 / 2, f'$z_2={z2:.0f}$ m', fontsize=9, color='#9C27B0', va='center')
    
    ax1.set_yticks(np.arange(-5, z2 + 8, max(5, int(z2/8))))
    
    # ══════════════════════════════════════════════════════════
    # DIAGRAMA 2: LÍNEAS DE ENERGÍA (EGL y HGL)
    # ══════════════════════════════════════════════════════════
    ax2.set_facecolor('#f0f4e8')
    
    # Posiciones x de los puntos clave
    x_pts = [0, 1.5, 1.5, 10]
    labels_x = ['Estanque\n(1)', 'Entrada\nbomba', 'Salida\nbomba', 'Cerro\n(2)']
    
    EGL_pts = [EGL_0, EGL_1, EGL_2, EGL_3]
    HGL_pts = [HGL_0, HGL_1, HGL_2, HGL_3]
    Z_pts   = [z1, -2, -2, z2]  # cotas del terreno en esos puntos
    
    # Relleno entre EGL y HGL (altura cinética)
    ax2.fill_between([0, 1.5], [EGL_0, EGL_1], [HGL_0, HGL_1],
                     alpha=0.25, color='#FF9800', label='$v^2/2g$ (alt. cinética)')
    ax2.fill_between([1.5, 10], [EGL_2, EGL_3], [HGL_2, HGL_3],
                     alpha=0.25, color='#FF9800')
    
    # Relleno entre HGL y cota (altura de presión)
    ax2.fill_between([0, 1.5], [HGL_0, HGL_1], [Z_pts[0], Z_pts[1]],
                     alpha=0.2, color='#2196F3', label='$P/\\rho g$ (alt. de presión)')
    ax2.fill_between([1.5, 10], [HGL_2, HGL_3], [Z_pts[2], Z_pts[3]],
                     alpha=0.2, color='#2196F3')
    
    # Cota (z)
    ax2.fill_between([0, 1.5], [Z_pts[0], Z_pts[1]], [0, 0],
                     where=[Z_pts[0] >= 0, Z_pts[1] >= 0],
                     alpha=0.3, color='#795548', label='$z$ (alt. potencial)')
    ax2.fill_between([1.5, 10], [Z_pts[2], Z_pts[3]], 0,
                     alpha=0.3, color='#795548')
    
    # Línea de cota (terreno)
    ax2.plot([0, 1.5, 1.5, 10], [z1, -2, -2, z2],
             'k--', linewidth=1.5, alpha=0.5, label='Cota terreno')
    
    # HGL
    ax2.plot([0, 1.5], [HGL_0, HGL_1], 'b-', linewidth=2.5, label='HGL (línea piezométrica)')
    ax2.plot([1.5, 10], [HGL_2, HGL_3], 'b-', linewidth=2.5)
    ax2.plot([1.5, 1.5], [HGL_1, HGL_2], 'b-', linewidth=2.5)
    
    # EGL
    ax2.plot([0, 1.5], [EGL_0, EGL_1], 'r-', linewidth=2.5, label='EGL (línea de energía)')
    ax2.plot([1.5, 10], [EGL_2, EGL_3], 'r-', linewidth=2.5)
    ax2.plot([1.5, 1.5], [EGL_1, EGL_2], 'r-', linewidth=2.5)
    
    # Puntos clave
    for xi, ei, hi in zip([0, 1.5, 1.5, 10], EGL_pts, HGL_pts):
        ax2.plot(xi, ei, 'ro', markersize=8, zorder=10)
        ax2.plot(xi, hi, 'bo', markersize=8, zorder=10)
    
    # Anotación de la bomba
    ax2.annotate('', xy=(1.5, EGL_2), xytext=(1.5, EGL_1),
                 arrowprops=dict(arrowstyle='->', color='#FF5722', lw=2.5))
    ax2.text(1.7, (EGL_1 + EGL_2) / 2, f'$H_b={Hb:.1f}$ m',
             fontsize=9, color='#BF360C', fontweight='bold')
    
    # Anotación pérdidas
    ax2.annotate('', xy=(10, EGL_3), xytext=(10, EGL_2 - (EGL_2 - EGL_3) * 0),
                 arrowprops=dict(arrowstyle='->', color='gray', lw=1.5))
    
    ax2.set_title('📊 Líneas de Energía (EGL) y Piezométrica (HGL)', fontsize=12, fontweight='bold', pad=10)
    ax2.set_xlabel('Posición en el sistema (esquemático)', fontsize=10)
    ax2.set_ylabel('Altura de energía [m]', fontsize=10)
    ax2.set_xticks([0, 1.5, 10])
    ax2.set_xticklabels(['Estanque (1)', 'Bomba', 'Cerro (2)'], fontsize=9)
    ax2.legend(loc='upper left', fontsize=8, framealpha=0.9)
    ax2.grid(True, alpha=0.3, linestyle='--')
    
    # ══════════════════════════════════════════════════════════
    # DIAGRAMA 3: BALANCE DE ENERGÍA (barras apiladas)
    # ══════════════════════════════════════════════════════════
    ax3.set_facecolor('#f5f5f5')
    
    categorias = ['Estanque (1)', 'Antes de\nBomba', 'Después de\nBomba', 'Cerro (2)']
    z_vals  = [z1,  -2,   -2,   z2]
    hv_vals = [hv1, hv2, hv2,  hv2]
    hp_vals = [P1_man, EGL_1 - (-2) - hv2, EGL_2 - (-2) - hv2, P2_man]
    
    x = np.arange(len(categorias))
    width = 0.45
    
    # Barras apiladas
    z_plot  = np.maximum(z_vals, 0)  # solo positivos
    bars1 = ax3.bar(x, z_plot, width, label='Cota z [m]', color='#795548', alpha=0.8)
    bars2 = ax3.bar(x, np.maximum(hp_vals, 0), width, bottom=z_plot,
                    label='Presión P/ρg [m]', color='#2196F3', alpha=0.8)
    bars3 = ax3.bar(x, hv_vals, width,
                    bottom=[z_plot[i] + max(hp_vals[i], 0) for i in range(len(x))],
                    label='Cinética v²/2g [m]', color='#FF9800', alpha=0.8)
    
    # Línea de energía total
    energias = [EGL_0, EGL_1, EGL_2, EGL_3]
    ax3.plot(x, energias, 'r-o', linewidth=2.5, markersize=8, label='Energía total H [m]', zorder=10)
    
    # Valores sobre las barras
    for xi, ei in zip(x, energias):
        ax3.text(xi, ei + 0.3, f'{ei:.1f} m', ha='center', fontsize=9,
                 fontweight='bold', color='darkred')
    
    ax3.set_title('⚡ Balance de Energía por Componente en cada Punto', fontsize=12, fontweight='bold', pad=8)
    ax3.set_ylabel('Altura de energía [m]', fontsize=10)
    ax3.set_xticks(x)
    ax3.set_xticklabels(categorias, fontsize=10)
    ax3.legend(loc='upper right', fontsize=9, framealpha=0.9)
    ax3.grid(True, axis='y', alpha=0.3, linestyle='--')
    ax3.set_facecolor('#f5f5f5')
    
    # ══════════════════════════════════════════════════════════
    # TÍTULO GENERAL Y RESULTADOS
    # ══════════════════════════════════════════════════════════
    resultado_texto = (
        f"PARÁMETROS DE ENTRADA  →  "
        f"z₂ = {z2:.0f} m  |  v = {v2:.1f} m/s  |  D = {D_mm:.0f} mm  |  hf = {hf:.1f} m     "
        f"RESULTADOS  →  "
        f"Hb = {Hb:.2f} m  |  Q = {Q_lps:.2f} L/s  |  P_bomba = {Pb_kW:.3f} kW  |  v²/2g = {hv2:.3f} m"
    )
    fig.text(0.5, 0.965, '💧 Sistema de Bombeo — Ecuación de Bernoulli Generalizada',
             ha='center', fontsize=14, fontweight='bold', color='#1A237E')
    fig.text(0.5, 0.945, resultado_texto,
             ha='center', fontsize=9, color='#37474F',
             bbox=dict(boxstyle='round,pad=0.4', facecolor='#E3F2FD', alpha=0.9))
    
    plt.show()
    
    # Imprimir ecuación resuelta
    print("\n" + "═"*65)
    print("  SOLUCIÓN DE LA ECUACIÓN DE BERNOULLI GENERALIZADA")
    print("═"*65)
    print(f"  Hb = z₂ + v₂²/2g + hf")
    print(f"  Hb = {z2:.1f} + {hv2:.3f} + {hf:.1f}")
    print(f"  Hb = {Hb:.3f} m")
    print(f"  Q  = A·v = π·({D_mm:.0f}mm)²/4 · {v2:.1f} m/s = {Q_lps:.4f} L/s")
    print(f"  P  = ρ·g·Q·Hb = {Pb_kW:.4f} kW")
    print("═"*65)

---
## 🎚️ Panel de Control Interactivo

Mueve los sliders para ver cómo cambia el sistema en tiempo real:

In [ ]:
# ============================================================
# 🎚️ SLIDERS INTERACTIVOS
# ============================================================

estilo = Layout(width='500px')

interact(
    graficar_bernoulli,
    z2   = FloatSlider(value=30, min=5,  max=100, step=1,   description='🏔️ Altura cerro z₂ [m]:',    layout=estilo, style={'description_width': '210px'}),
    v2   = FloatSlider(value=2,  min=0.1,max=8,   step=0.1, description='💨 Velocidad v [m/s]:',       layout=estilo, style={'description_width': '210px'}),
    D_mm = FloatSlider(value=100,min=25, max=300, step=5,   description='🔩 Diámetro D [mm]:',         layout=estilo, style={'description_width': '210px'}),
    hf   = FloatSlider(value=5,  min=0,  max=30,  step=0.5, description='⚡ Pérdidas de carga hf [m]:',layout=estilo, style={'description_width': '210px'})
);

---
## 📝 Ejercicios Propuestos

Usa la simulación interactiva para responder las siguientes preguntas:

### Ejercicio 1 — Efecto de la altura
> Con $v = 2$ m/s, $D = 100$ mm y $h_f = 5$ m, varía la altura $z_2$ entre 10 m y 80 m.  
> **¿Cómo cambia $H_b$? ¿La relación es lineal?** Explica por qué.

### Ejercicio 2 — Efecto del diámetro
> Fija la altura $z_2 = 40$ m y las pérdidas $h_f = 5$ m. Varía el diámetro entre 50 mm y 200 mm, manteniendo $v = 3$ m/s.  
> **¿Cómo afecta el diámetro al caudal $Q$ y a la potencia requerida?**

### Ejercicio 3 — Pérdidas de carga
> Con $z_2 = 50$ m, $v = 2$ m/s, $D = 150$ mm, incrementa $h_f$ de 0 a 20 m.  
> **¿Qué porcentaje de $H_b$ representan las pérdidas cuando $h_f = 20$ m?**

### Ejercicio 4 — Diseño de un sistema real ⭐
> Una ciudad requiere abastecer a un sector ubicado 65 m sobre el nivel del estanque. El caudal mínimo es 8 L/s y se estiman pérdidas de 12 m.  
> **Determina el diámetro de tubería adecuado y la potencia mínima de la bomba.**  
> *Pista: Recuerda que $Q = A \cdot v$ y que velocidades entre 0.5 y 3 m/s son recomendables.*

---
## 📖 Referencias

- Munson, B.R., Young, D.F., & Okiishi, T.H. — *Fundamentals of Fluid Mechanics*
- Cengel, Y.A. & Cimbala, J.M. — *Mecánica de Fluidos: Fundamentos y Aplicaciones*
- Streeter, V.L. & Wylie, E.B. — *Mecánica de Fluidos*

---
*Notebook desarrollado con Python · matplotlib · ipywidgets*